# Project
- Work solo or with one other person
- Topic should be something interesting and fun for you
- Your code should be written in Python 3.6
- Effort roughly equivalent to two homeworks
    - perhaps 150-200 lines of code
    - sometimes people really get into it, which is fine
        - have seen 1,000 line programs, but this is NOT necessary
- All I'm really looking for is to see you've learned some Python and can do something with it
- Happy to discuss your project with you if you have questions
- Hangman games are discouraged(seen a pile of them)

# If you did a solo project, fill in this cell

```
name:
UNI: 
```

Submit a zip file named 'yourUNI.zip' to CW2 that includes this notebook, your source, documentation, and data files. The zip must expand into a directory named 'YourUNI', and all your files should be under the YourUNI directory.


# If you did a project with a partner, fill in this cell

```
name1:
UNI1:

name2:
UNI2:

roughly, who did what:

```

Each of you should submit the SAME zip file named   
'UNI1-UNI2.zip' to CW2. The zip should include this notebook, your source, documentation, and data files. The zip must expand into a directory named 'UNI1-UNI2', and all your files should be under the UNI1-UNI2 directory.


# What does your project do?

...



# What modules did you use, if any?

......

# Explain how we can run your project
- Make sure you include all python and data files
- If your project has graphics, include some snapshots
- Any documentation should be written in a notebook
    - do NOT submit ASCII/plain text, PDFs, Word files, etc.
- If for some reason it is not feasible for us to run your project, make an appointment with Larry to demo it


.....


# An evaluation of Python's suitability to your task
    - Were any problems encountered?
    - Was Python a good fit for your project?
        - In retrospect, would you have preferred to use another language for your project - R, Java, Matlab, etc
        - Be honest! You don't have to praise Python. I want to hear about your actual experience. 
        

.....

In [3]:
from selenium import webdriver
import requests
import time

def save_page(url, out_file, wait_for_class='search-card'):
    """ Saves a page, using the Selenium browser testing framework to run any Javascript
        , writing the resulting final DOM as HTML source to disk.
        Waits until element of a given class is loaded to make sure Javascript has actually
        completed before source is retrieved. """

    browser = webdriver.Chrome()
    browser.get(url)
    # Wait until elment with given class is loaded
    while True:
        try:
            # if the element cannot be found, this will throw
            browser.find_element_by_class_name(wait_for_class)
            break # otherwise the loop will end here
        except:
            pass # do nothing, try again
    # source has loaded - write out result
    open(out_file, 'w').write(browser.page_source)

#loop through 42 pages of search results & save to working directory
page = 0    
while page <= 41:
    url = 'https://www.vitals.com/search?display_type=Doctor&state=NY&page=' + str(page)
    page += 1
    save_page(url, '/home/katya/Documents/Py Final Project/Pages/Vitals' + str(page) + ".html")

In [92]:
import re
from bs4 import BeautifulSoup

def parse_page(filepath):
    page_contents = open(filepath).read().strip()
    my_soup = BeautifulSoup(page_contents, "html.parser")
    data = []
    for card in my_soup.findAll("div", attrs = {"class": "search-card"}):
        name_element = card.find("span", attrs = {"class": "name"})
        if name_element == None:
            continue
        name = name_element.text.strip()
        specialty_element = card.find("span", attrs = {"class": "specialty"})
        if specialty_element == None:
            continue
        specialty = specialty_element.text.strip().split(',')[0]
        location_element = card.find("span", attrs = {"class": "address"})
        if location_element == None:
            continue
        location = location_element.text.strip().split(',')[0]
        rating_element = card.find("span", attrs = {"class": "rating-text"})
        if rating_element == None:
            continue
        rating = rating_element.text.strip('')
        reviews_element = str(card.select("a[href*=reviews]"))
        if reviews_element == None:
            continue
        reviews = ''.join(re.findall(r'\d+', reviews_element))
        row = (name, specialty, location, rating, reviews)
        data.append(row)
        
    return (data)
    


In [93]:
import csv

with open('healthreviews.csv','w') as csvfile:
    writer = csv.writer(csvfile)
    titlerow = ("Name", "Specialty", "Location", "Rating", "Reviews")
    writer.writerow(titlerow)
    for page in range(1, 43):
        all_rows = parse_page("/home/katya/Documents/Py Final Project/Pages/Vitals" + str(page) + ".html")
        for row in all_rows:
            writer.writerow(row)
        